In [1]:
import onnxruntime as ort
import numpy as np
import os
from PIL import Image
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
TEST_DIR = "skin_cancer/test" 
ORIGINAL_MODEL = "skin_cancer_resnet.onnx"
QUANTIZED_MODEL = "skin_cancer_mobile_quant.onnx"
IMG_SIZE = (224, 224)

In [3]:
session_float = ort.InferenceSession(ORIGINAL_MODEL)
session_quant = ort.InferenceSession(QUANTIZED_MODEL)

In [8]:
input_name = session_float.get_inputs()[0].name
input_name

'input_image'

In [4]:
classes = sorted([d for d in os.listdir(TEST_DIR) if os.path.isdir(os.path.join(TEST_DIR, d))])
class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
print(f"Classes found: {classes}")

Classes found: ['Benign keratosis-like lesions', 'Melanocytic nevi', 'Melanoma']


In [5]:
def preprocess_image(image_path):
    """
    Load image, resize to 224x224, convert to array.

    """
    img = Image.open(image_path).convert('RGB')
    img = img.resize(IMG_SIZE)
    img_data = np.array(img).astype(np.float32)
    
    # Add Batch Dimension: (224, 224, 3) -> (1, 224, 224, 3)
    img_data = np.expand_dims(img_data, axis=0)
    return img_data

In [6]:
y_true = []
y_pred_float = []
y_pred_quant = []
total_files = sum([len(files) for r, d, files in os.walk(TEST_DIR)])
count = 0

In [9]:
for class_name in classes:
    class_dir = os.path.join(TEST_DIR, class_name)
    ground_truth_id = class_to_idx[class_name]
    
    for fname in os.listdir(class_dir):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
            
        # Prepare Data
        img_path = os.path.join(class_dir, fname)
        input_data = preprocess_image(img_path)
        
        # Run Original
        out_float = session_float.run(None, {input_name: input_data})[0]
        pred_float = np.argmax(out_float)
        
        # Run Quantized
        out_quant = session_quant.run(None, {input_name: input_data})[0]
        pred_quant = np.argmax(out_quant)
        
        # Store results
        y_true.append(ground_truth_id)
        y_pred_float.append(pred_float)
        y_pred_quant.append(pred_quant)
        
        count += 1
        if count % 10 == 0:
            print(f"Processed {count}/{total_files} images...", end='\r')

In [10]:
acc_float = accuracy_score(y_true, y_pred_float)
acc_quant = accuracy_score(y_true, y_pred_quant)

print(f"Original Model Accuracy:  {acc_float * 100:.2f}%")
print(f"Quantized Model Accuracy: {acc_quant * 100:.2f}%")

Original Model Accuracy:  75.15%
Quantized Model Accuracy: 73.33%


In [11]:
drop = acc_float - acc_quant
print(f"Accuracy Drop:            {drop * 100:.2f}%")

Accuracy Drop:            1.82%


In [12]:
# Show specific cases where they disagreed
disagreements = 0
for i in range(len(y_true)):
    if y_pred_float[i] != y_pred_quant[i]:
        disagreements += 1
        print(f"Img {i}: Original said {classes[y_pred_float[i]]}, Quant said {classes[y_pred_quant[i]]}")

print(f"The models disagreed on {disagreements} images out of {total_files}.")

Img 26: Original said Melanocytic nevi, Quant said Benign keratosis-like lesions
Img 39: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 84: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 87: Original said Benign keratosis-like lesions, Quant said Melanoma
Img 99: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 104: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 221: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 225: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 240: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 257: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 259: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 274: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 276: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 282: Original said Melanoma, Qu

VERDICT: Safe to deploy! ✅
